# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp


In [7]:
from astropy.time import Time
from astropy.utils.data import download_file

from jorbit.data.constants import JORBIT_EPHEM_URL_BASE
from jorbit.mpchecker.parse_jorbit_ephem import (
    load_mpcorb,
)

x0 = jnp.load(download_file(JORBIT_EPHEM_URL_BASE + "x0.npy", cache=True))
names = jnp.load(download_file(JORBIT_EPHEM_URL_BASE + "names.npy", cache=True))
m = load_mpcorb()
ind=200
chunk = jnp.load(
            download_file(
                JORBIT_EPHEM_URL_BASE + f"chebyshev_coeffs_fwd_{ind:03d}.npy",
                cache=True,
            )
        )

In [4]:
x0.shape, names.shape, chunk.shape, len(m)

((1424291, 3), (1424291,), (1424291, 11, 2), 1436009)

In [5]:
jnp.unique(x0[:,0]).shape

(1386562,)

In [5]:
import astropy.units as u
import numpy as np
from astropy.coordinates import SkyCoord

from jorbit.mpchecker import nearest_asteroid

start_time = Time("2021-08-20") - 4*u.hour
end_time = Time("2021-09-16") + 28*u.hour
times1 = Time(np.arange(start_time.jd, end_time.jd, 1*u.min.to(u.day)), format='jd')[::10]


separations1, asteroids1, flags1 = nearest_asteroid(
    coordinate=SkyCoord(349.3783*u.deg, 8.2131*u.deg),
    times=times1,
    precomputed=None,
    radius=3*u.arcmin,
    compute_contamination=False,
)
len(asteroids1)

23

In [6]:
asteroids1["Packed designation"]

G7249
O5229
34138
c7876
h3330
h7909
l5274
s2387
v3019
x7234
K03G60U


In [8]:
# both of these are present... and digging into x0, they have identical states...
# K12TB3C
# K12Tb3C

In [9]:
# they're both in the MPCORB.DAT file...
(m["Packed designation"] == "K12TB3C").sum(), (m["Packed designation"] == "K12Tb3C").sum()

(1, 1)

In [10]:
# they're definitely different objects...
# K12Tb3C 17.53  0.15 K24AH  68.55789  216.20583  137.98804    5.27906  0.2077642  0.17414785   3.1758304  1 MPO781609    57   4 2006-2023 0.14 M-v 3Ek MPCLINUX   0000          2012 TC373         20231019
# K12TB3C 19.45  0.15 K24AH 204.06890  202.11531  215.61529    8.68561  0.3106434  0.21452971   2.7636164  3 MPO689324    19   2 2012-2021 0.31 M-v 3Ek Veres      0000          2012 TC113         20211006

In [8]:
# why do they have identical x0, v0? because Horizons is resolving both to the same object
# K12Tb3C = 2012 TC373
# K12TB3C = 2012 TC113
# Horizons is resolving both to 2012 TC113

from astroquery.jplhorizons import Horizons

t0 = Time("2020-01-01")


particle_name = "K12TB3C"
obj1 = Horizons(
                id=particle_name,
                location="500@0",
                epochs=t0.tdb.jd,
                id_type="smallbody",
            )
vecs1 = obj1.vectors(refplane="earth")

particle_name = "K12Tb3C"
obj2 = Horizons(
                id=particle_name,
                location="500@0",
                epochs=t0.tdb.jd,
                id_type="smallbody",
            )
vecs2 = obj2.vectors(refplane="earth")

vecs1, vecs2

(<Table masked=True length=1>
  targetname    datetime_jd    ...       range             range_rate     
     ---             d         ...         AU                AU / d       
    str12         float64      ...      float64             float64       
 ------------ ---------------- ... ----------------- ---------------------
 (2012 TC113) 2458849.50080074 ... 3.614521415843135 -0.000287058740770556,
 <Table masked=True length=1>
  targetname    datetime_jd    ...       range             range_rate     
     ---             d         ...         AU                AU / d       
    str12         float64      ...      float64             float64       
 ------------ ---------------- ... ----------------- ---------------------
 (2012 TC113) 2458849.50080074 ... 3.614521415843135 -0.000287058740770556)

In [9]:
str(obj1), str(obj2)

('JPLHorizons instance "K12TB3C"; location=500@0, epochs=[np.float64(2458849.5008007395)], id_type=smallbody',
 'JPLHorizons instance "K12Tb3C"; location=500@0, epochs=[np.float64(2458849.5008007395)], id_type=smallbody')

In [ ]:
# It's not astroquery though- it's Horizons itself?
# (now I've patched the bulk_query, so it looks different but was identical before)

from astropy.time import Time

from jorbit.utils.horizons import horizons_bulk_vector_query

v1 = horizons_bulk_vector_query(
    target="K12TB3C",
    center="500@0",
    times=Time("2020-01-01"),
    disable_astroquery=True,
)
v2 = horizons_bulk_vector_query(
    target="K12Tb3C",
    center="500@0",
    times=Time("2020-01-01"),
    disable_astroquery=True,
)
v1, v2

(         JD_TDB                              Cal         x         y  \
 0  2.458850e+06   A.D. 2020-Jan-01 00:01:09.1839 -1.691344 -3.019393   
 
           z        vx        vy        vz        LT        RG        RR  
 0 -1.042777  0.006722 -0.003302 -0.000346  0.020876  3.614521 -0.000287  ,
          JD_TDB                              Cal         x         y  \
 0  2.458850e+06   A.D. 2020-Jan-01 00:01:09.1839 -2.539567  2.376144   
 
           z        vx        vy        vz        LT        RG        RR  
 0  1.007744 -0.006806 -0.004783 -0.001251  0.020913  3.620912  0.001286  )

In [ ]:
# when we search their unpacked designations, we get different results though

v1 = horizons_bulk_vector_query(
    target="2012 TC373",
    center="500@0",
    times=Time("2020-01-01"),
    disable_astroquery=True,
)
v2 = horizons_bulk_vector_query(
    target="2012 TC113",
    center="500@0",
    times=Time("2020-01-01"),
    disable_astroquery=True,
)
v1, v2

(         JD_TDB                              Cal         x         y  \
 0  2.458850e+06   A.D. 2020-Jan-01 00:01:09.1839 -2.539567  2.376144   
 
           z        vx        vy        vz        LT        RG        RR  
 0  1.007744 -0.006806 -0.004783 -0.001251  0.020913  3.620912  0.001286  ,
          JD_TDB                              Cal         x         y  \
 0  2.458850e+06   A.D. 2020-Jan-01 00:01:09.1839 -1.691344 -3.019393   
 
           z        vx        vy        vz        LT        RG        RR  
 0 -1.042777  0.006722 -0.003302 -0.000346  0.020876  3.614521 -0.000287  )

In [ ]:
# same for astroquery

particle_name = "2012 TC373"
obj1 = Horizons(
                id=particle_name,
                location="500@0",
                epochs=t0.tdb.jd,
                id_type="smallbody",
            )
vecs1 = obj1.vectors(refplane="earth")

particle_name = "2012 TC113"
obj2 = Horizons(
                id=particle_name,
                location="500@0",
                epochs=t0.tdb.jd,
                id_type="smallbody",
            )
vecs2 = obj2.vectors(refplane="earth")

vecs1, vecs2

(<Table masked=True length=1>
  targetname    datetime_jd    ...       range            range_rate     
     ---             d         ...         AU               AU / d       
    str12         float64      ...      float64            float64       
 ------------ ---------------- ... ----------------- --------------------
 (2012 TC373) 2458849.50080074 ... 3.620912321991971 0.001286486479685278,
 <Table masked=True length=1>
  targetname    datetime_jd    ...       range             range_rate     
     ---             d         ...         AU                AU / d       
    str12         float64      ...      float64             float64       
 ------------ ---------------- ... ----------------- ---------------------
 (2012 TC113) 2458849.50080074 ... 3.614521415843135 -0.000287058740770556)

In [3]:
from jorbit.utils.horizons import (
    HorizonsQueryConfig,
    horizons_query_context,
    make_horizons_request,
    parse_horizons_response,
)

center="500@0"
query_type = "VECTOR"
skip_daylight = False
times = [Time("2020-01-01")]

# target = "K12TB3C"
target="K12Tb3C"
# target = "00001"

lines = [
    "!$$SOF",
    f'COMMAND= "DES={target};"',
    "OBJ_DATA='NO'",
    "MAKE_EPHEM='YES'",
    f"CENTER='{center}'",
    "REF_PLANE='FRAME'",
    "CSV_FORMAT='YES'",
    "OUT_UNITS='AU-D'",
    "CAL_FORMAT='JD'",
    "TLIST_TYPE='JD'",
]

if query_type == "VECTOR":
    lines.append("TABLE_TYPE='VECTOR'")
elif query_type == "OBSERVER":
    lines.extend(
        [
            "TABLE_TYPE='OBSERVER'",
            "QUANTITIES='1,36,37'",
            "ANG_FORMAT='DEG'",
            "EXTRA_PREC = 'YES'",
        ]
    )
    if skip_daylight:
        lines.append("SKIP_DAYLT = 'YES'")

lines.append("TLIST=")
for t in times:
    if query_type == "VECTOR":
        time_value = t.tdb.jd if isinstance(t, Time) else t
    elif query_type == "OBSERVER":
        time_value = t.utc.jd if isinstance(t, Time) else t
    lines.append(f"'{time_value}'")

query = "\n".join(lines)

with horizons_query_context(query) as query_content:
    response_text = make_horizons_request(query_content)
    res = parse_horizons_response(
        response_text, HorizonsQueryConfig.VECTOR_COLUMNS
    )

res["x"].iloc[0]

np.float64(-2.539566784607915)

In [60]:
response_text

'API VERSION: 1.0\nAPI SOURCE: NASA/JPL Horizons API\n\n\n\n*******************************************************************************\nEphemeris / WWW_USER Fri Feb 21 11:59:31 2025 Pasadena, USA      / Horizons\n*******************************************************************************\nTarget body name: (2012 TC113)                    {source: JPL#1}\nCenter body name: Solar System Barycenter (0)     {source: DE441}\nCenter-site name: BODY CENTER\n*******************************************************************************\nStart time      : A.D. 2020-Jan-01 00:01:09.1839 TDB\nStop  time      : A.D. 2020-Jan-01 00:01:09.1839 TDB\nStep-size       : DISCRETE TIME-LIST\n*******************************************************************************\nCenter geodetic : 0.0, 0.0, 0.0                   {E-lon(deg),Lat(deg),Alt(km)}\nCenter cylindric: 0.0, 0.0, 0.0                   {E-lon(deg),Dxy(km),Dz(km)}\nCenter radii    : (undefined)\nSmall perturbers: Yes                

In [27]:
# K12Tb3C = 2012 TC373
# K12TB3C = 2012 TC113
obj = Horizons(
                id="2012 TC373",
                location="500@0",
                epochs=t0.tdb.jd,
                id_type="smallbody",
            )
vecs = obj.vectors(refplane="earth")
vecs["x"]

-2.539566784607915
